In [1]:
from model import Chatbot, Encoder, Decoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from pathlib import Path
from unidecode import unidecode
import numpy as np
import pickle

In [2]:
data_dir = Path('data')

with open(data_dir/'preguntas.txt', 'r', encoding='utf-8') as f:
    questions = f.readlines()

with open(data_dir/'respuestas.txt', 'r', encoding='utf-8') as f:
    answers = f.readlines()

print(f"El archivo 'preguntas.txt' contiene {len(questions)} preguntas")

El archivo 'preguntas.txt' contiene 273 preguntas


In [3]:
START_TOKEN = '<START> '
END_TOKEN = ' <END>'

processed_questions = list(map(lambda x: unidecode(x.lower()), questions))
processed_answers = list(map(lambda x: START_TOKEN + unidecode(x.lower().strip()) + END_TOKEN, answers))

print(processed_answers[0])

<START> "rosebud" <END>


In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_questions + processed_answers)
vocab_size = len(tokenizer.word_index)

print(f"There are {vocab_size} unique words")

There are 1725 unique words


In [5]:
# Preparing Encoder Input
tokenized_questions = tokenizer.texts_to_sequences(processed_questions)
max_input_length = max(list(map(len, tokenizer.texts_to_sequences(processed_questions))))
encoder_input = np.array(pad_sequences(tokenized_questions, maxlen=max_input_length, padding='post'))
print(f"Encoder input data shape: {encoder_input.shape}")

# Preparing Decoder Input
tokenized_answers = tokenizer.texts_to_sequences(processed_answers)
max_output_length = max(list(map(len, tokenizer.texts_to_sequences(processed_answers))))
decoder_input = np.array(pad_sequences(tokenized_answers, maxlen=max_input_length, padding='post'))
print(f"Decoder input data shape: {decoder_input.shape}")

# Preparing Target Output
tokenized_output = list(map(lambda x: x[1:], tokenized_answers))
padded_target = pad_sequences(tokenized_output, maxlen=max_input_length, padding='post')
decoder_target = np.array(to_categorical(padded_target))
print(f"Decoder target data shape: {decoder_target.shape}")

with open(data_dir/'tokenizer.pkl', 'wb') as f:
    pickle.dump({
        'tokenizer': tokenizer,
        'vocab_size': vocab_size,
        'max_input_length': max_input_length,
        'max_output_length': max_output_length,
        'model_bootstrap': (encoder_input[:1], decoder_input[:1], decoder_target[:1])   
    }, f, protocol=pickle.DEFAULT_PROTOCOL)

Encoder input data shape: (273, 31)
Decoder input data shape: (273, 31)
Decoder target data shape: (273, 31, 1726)


In [12]:
model = Chatbot(vocab_size + 1)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input, decoder_input], decoder_target, batch_size=10, epochs=100)

Epoch 1/100
28/28 [==============================] - 15s 198ms/step - loss: 1.9370 - accuracy: 0.1519
Epoch 2/100
28/28 [==============================] - 6s 202ms/step - loss: 1.4041 - accuracy: 0.1822
Epoch 3/100
28/28 [==============================] - 6s 199ms/step - loss: 1.3133 - accuracy: 0.2297
Epoch 4/100
28/28 [==============================] - 6s 198ms/step - loss: 1.2644 - accuracy: 0.2486
Epoch 5/100
28/28 [==============================] - 6s 197ms/step - loss: 1.0438 - accuracy: 0.2968
Epoch 6/100
28/28 [==============================] - 5s 195ms/step - loss: 1.2076 - accuracy: 0.2716
Epoch 7/100
28/28 [==============================] - 5s 194ms/step - loss: 0.9944 - accuracy: 0.3114
Epoch 8/100
28/28 [==============================] - 6s 198ms/step - loss: 1.0604 - accuracy: 0.3073
Epoch 9/100
28/28 [==============================] - 5s 195ms/step - loss: 0.9691 - accuracy: 0.3250
Epoch 10/100
28/28 [==============================] - 6s 202ms/step - loss: 1.0103 - accur

In [13]:
model.save_weights(Path('model')/'model.hdf5')

In [14]:
def preprocess_input(text: str):
    text = text.lower()
    text = unidecode(text)
    tokenized_text = tokenizer.texts_to_sequences([text])
    padded_text = pad_sequences(tokenized_text, maxlen=max_input_length, padding='post')

    return padded_text

In [15]:
#model = Chatbot(vocab_size + 1)
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.train_on_batch([encoder_input[:1], decoder_input[:1]], decoder_target[:1])
#model.load_weights(Path('model')/'model.hdf5')

In [16]:
question = 'Qué pelicula de Alfred Hitchcock se rodo como si fuera una toma continua'
empty_target_seq = np.array([[tokenizer.word_index['start']]])
stop_condition = False
answer = ''
stop_words = ['adios', 'gracias']

proc_question = preprocess_input(question)

h, c = model.encoder.predict(proc_question)

while not stop_condition:
    decoder_output, h, c = model.decoder.predict([empty_target_seq, h, c])
    sampled_word_index = np.argmax( decoder_output[0, -1, :] )
    sampled_word = tokenizer.index_word.get(sampled_word_index, None)
    if sampled_word == 'end' or len(answer.split()) > max_output_length:
        stop_condition = True
    else:
        answer += f"{sampled_word} "
    

    empty_target_seq = np.array([[sampled_word_index]])
    enc_stats = [h, c]


print(answer)


la soga 
